<a href="https://colab.research.google.com/github/MikaTina/Project_ML-SII/blob/main/GPT2_Sentiment_on_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code is based on https://medium.com/analytics-vidhya/gpt2-for-sentiment-analysis-38cd9832d5e9

Modified to use the full IMDB of movie reviews instead of the 50 examples file.

# **Import Libraries**
Tensorflow 1.x is used because of dependencies needed with gpt_2_simple.

In [ ]:
%tensorflow_version 1.x
!pip install gpt-2-simple 
import json
import pandas as pd
import os
import numpy as np
import tensorflow as tf
import gpt_2_simple as gpt2

TensorFlow 1.x selected.
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.2-cp37-none-any.whl size=23621 sha256=553f02efdb778b37be299d4fd7bfdbf4a07e285b4669cb9c483f6ad9dde2d88b
  Stored in directory: /root/.cache/pip/wheels/68/1d/15/c87a4302a6c7273ce1b4f282bec3c6877fb2f521535d87d30f
Successfully built gpt-2-simple
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# **Import Dataset**

In [ ]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

84131840/84125825 [==============================] - 1s 0us/step


In [ ]:
# The shutil module offers a number of high-level 
# operations on files and collections of files.
import os
import shutil
# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Create sub directory path ("/aclImdb/test")
test_dir = os.path.join(main_dir, 'test')
# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
# View the final train folder
print(os.listdir(train_dir))
# View the final test folder
print(os.listdir(test_dir))

['urls_neg.txt', 'urls_unsup.txt', 'pos', 'labeledBow.feat', 'neg', 'unsupBow.feat', 'urls_pos.txt']
['urls_neg.txt', 'pos', 'labeledBow.feat', 'neg', 'urls_pos.txt']


# **Preprocessing**
The dataset chosed has tre principal folders: one contains files of positive reviews, one that contains files of negative reviews and a file of unlabeled reviews.
We chose to exclude the unlabeled reviews and divide the labeled reviews in train and test sets, with no validation set. The objective is to hope that gpt2 learns to complete a sentence formatted as:

// *review* || This review is *label* 

where *label* should be completed by gpt. 

The initial preprocessing of the files is inspired by https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671




In [ ]:
dir_train_pos = os.listdir(train_dir + "/pos")
dir_train_neg = os.listdir(train_dir + "/neg")
dir_test_pos = os.listdir(test_dir + "/pos")
dir_test_neg = os.listdir(test_dir + "/neg")

In [ ]:
dataset = []
pos_train_dir = os.path.join(train_dir,'pos')
neg_train_dir = os.path.join(train_dir,'neg')
pos_test_dir = os.path.join(test_dir,'pos')
neg_test_dir = os.path.join(test_dir,'neg')

for element in dir_train_pos:
  input_file = open(pos_train_dir + "/"+ element, "r")
  line = input_file.readline()
  dataset.append("// " + line + " || This review is " + "Positive" + "\n")
  input_file.close()

for element in dir_train_neg:
  input_file = open(neg_train_dir + "/"+ element, "r")
  line = input_file.readline()
  dataset.append("// " + line + " || This review is " + "Negative" + "\n")
  input_file.close()

for element in dir_test_pos:
  input_file = open(pos_test_dir + "/"+ element, "r")
  line = input_file.readline()
  dataset.append("// " + line + " || This review is " + "Positive" + "\n")
  input_file.close()

for element in dir_test_neg:
  input_file = open(neg_test_dir + "/"+ element, "r")
  line = input_file.readline()
  dataset.append("// " + line + " || This review is " + "Negative" + "\n")
  input_file.close()

#dataset

In [ ]:
import random
random.shuffle(dataset)
#dataset

In [ ]:
len(dataset)

50000

In [ ]:
count = 0
train = []
while count < 40000:
  train.append(dataset[count])
  count = count + 1

count = 40000
test = []
while count < 50000:
    test.append(dataset[count])
    count = count + 1
#train

In [ ]:
#test

In [ ]:
file_train = open("train.txt", "w")
for element in train:
  file_train.write(element)
file_train.close()

train = []

In [ ]:
#useless
#file_test = open("test.txt", "w")
#for element in test:
#  file_test.write(element)
#file_test.close()

# **Downloading and Training the model**

In [ ]:
#model_name = "117M"
model_name = "345M" # The GPT-2 model we're using
#model_name = "774M" #OOM, it was fun until it lasted
gpt2.download_gpt2(model_name=model_name) # Download the model

Fetching checkpoint: 1.05Mit [00:00, 475Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.24Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 656Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [04:25, 5.35Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 317Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.23Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 4.13Mit/s]


In [ ]:
train_path = "train.txt"
sess = gpt2.start_tf_sess()
gpt2.finetune(sess, #session
              train_path, #path string of the file where the training set is saved
              model_name=model_name, #used model 
              run_name='run1', 
              checkpoint_dir='checkpoint', 
              steps=2)   # max number of training steps


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/345M/model.ckpt
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:11<00:00, 71.85s/it]


dataset has 12105267 tokens
Training...
[1 | 68.60] loss=3.15 avg=3.15
[2 | 125.43] loss=3.67 avg=3.41
Saving checkpoint/run1/model-2


# **Interaction with the model and results**
Code references that inspired this section:

https://github.com/minimaxir/gpt-2-simple/blob/master/gpt_2_simple/gpt_2.py

https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py

https://github.com/openai/gpt-2/blob/master/src/interactive_conditional_samples.py

https://colab.research.google.com/drive/1ulO-Z0G6BdvQAZ83PJNXCS0ygtRtp46g




In [ ]:
#Download library gpt2
!git clone https://github.com/openai/gpt-2.git
import os
os.chdir("gpt-2/src/")
import tensorflow as tf
import model, sample, encoder
os.chdir('../../')

Cloning into 'gpt-2'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 4.38 MiB | 12.12 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [ ]:
samples = open("samples.txt","w")
samples.write("Positive"+"\n")
samples.write("Negative"+"\n")
samples.close()

In [ ]:
correct = 0
n_tests = 5000
for element in test:
  print(element + "\n")
  split = element.split("|| This review is ")
  input = split[0].strip()
  #print(input + "\n")
  input = input + " ||  This review is "
  #print(input + "\n")
  expected_output = split[1].strip()
  gpt2.generate(sess,
             run_name='run1',
             checkpoint_dir='checkpoint',
             model_name=model_name,
             model_dir='models',
             sample_dir='samples',
             return_as_list=False,
             truncate=None,
             destination_path=None,
             sample_delim='\n',
             prefix=input,
             seed=None,
             nsamples=1,
             batch_size=1,
             length=8,
             temperature=0.7,
             top_k=0,
             top_p=0.0,
             include_prefix=True)

  print("Expected: " + expected_output + ", Answer: ") #+ str(out) + "\n")
  #if (str(out).casefold() == expected_output.casefold()):
    #correct = correct + 1
print("Correct " + str(correct) + " out of" + str(n_tests) + "\n")


// Horrible acting, horrible cast and cheap props. Would've been a lot better if was set as an action parody style movie. What a waste. Starting from the name of the movie.<br /><br />"The Enemy" Naming it "Action Movie" would've made it better. (contributing to the parody effect). The cop looking like a 60 Year old player, the blond girl just having the same blank boring look on her face at all times. Towards the end of the movie him and her are working together to take down the bad guys and every time they exchange words it just feels like the cheap lines given before a sex scene in a porn movie. Horrible. Don't waste your time. || This review is Negative


// Horrible acting, horrible cast and cheap props. Would've been a lot better if was set as an action parody style movie. What a waste. Starting from the name of the movie.<br /><br />"The Enemy" Naming it "Action Movie" would've made it better. (contributing to the parody effect). The cop looking like a 60 Year old player, the bl

In [ ]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

#        while True:
#            raw_text = input("Model prompt >>> ")
#            while not raw_text:
#                print('Prompt should not be empty!')
#                raw_text = input("Model prompt >>> ")
#            context_tokens = enc.encode(raw_text)
#            generated = 0
#            for _ in range(nsamples // batch_size):
#                out = sess.run(output, feed_dict={
#                    context: [context_tokens for _ in range(batch_size)]
#                })[:, len(context_tokens):]
#                for i in range(batch_size):
#                    generated += 1
#                    text = enc.decode(out[i])
#                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
#                    print(text)
#            print("=" * 80)
        n_tests = 5000
        correct = 0
        for element in test:
          split = element.split("||  This review is")
          input = split[0].strip()
          #print(input + "\n")
          input = input + " ||  This review is "
          print(input + "\n")
          expected_output = split[1].strip()
          context_tokens = enc.encode(input)
          #due to a bug https://github.com/minimaxir/gpt-2-simple/issues/38
          #long long reviews are excluded
          try: 
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    text = enc.decode(out[i])
                    print("Expected: " + expected_output + ", Answer: " + text + "\n")
                    if (text.casefold() == expected_output.casefold()):
                      correct = correct + 1
          except:
            print("-------------- too long -------------")
            n_tests = n_tests - 1
        print("Correct " + str(correct) + " out of" + str(n_tests) + "\n")

In [ ]:
#Test
!ls
interact_model(
    'run1',
    None,
    1,
    1,
    2,
    1,
    0,
    './checkpoint'
)